<a href="https://colab.research.google.com/github/KrsNovita2717/FaceExpressionDetection/blob/main/ExpressionRecognitionWebcam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import dependencies
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from base64 import b64decode, b64encode
import cv2
import numpy as np
import PIL
import io
import html
import time
import gdown
from keras.models import load_model

In [ ]:
# Unduh model
url = 'https://drive.google.com/uc?id=1BuVp5Te1vKMV062dJANtgf2o1ob7exZ4'
output = 'FaceExpressionDetection.h5'

gdown.download(url, output, quiet=False)

In [ ]:
# Memuat model untuk pengenalan ekspresi wajah
model = load_model('FaceExpressionDetection.h5')

In [ ]:
# Fungsi untuk mengonversi objek JavaScript menjadi gambar OpenCV
def js_to_image(js_reply):
    """
    Params:
            js_reply: Objek JavaScript yang berisi gambar dari webcam
    Returns:
            img: Gambar OpenCV BGR
    """
    # decode gambar base64
    image_bytes = b64decode(js_reply.split(',')[1])
    jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
    img = cv2.imdecode(jpg_as_np, flags=1)

    return img

In [ ]:
# Fungsi untuk mengonversi gambar bounding box OpenCV menjadi byte string base64 untuk overlay pada video stream
def bbox_to_bytes(bbox_array):
    """
    Params:
            bbox_array: Array Numpy (pixel) yang berisi bounding box untuk overlay pada video stream.
    Returns:
            bytes: Byte string gambar base64
    """
    bbox_PIL = PIL.Image.fromarray(bbox_array, 'RGBA')
    iobuf = io.BytesIO()
    bbox_PIL.save(iobuf, format='png')
    bbox_bytes = 'data:image/png;base64,{}'.format((str(b64encode(iobuf.getvalue()), 'utf-8')))

    return bbox_bytes

In [ ]:
# Inisialisasi model deteksi wajah dengan Haar Cascade
face_cascade = cv2.CascadeClassifier(cv2.samples.findFile(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'))

In [ ]:
# JavaScript untuk membuat live video stream menggunakan webcam sebagai input
def video_stream():
    js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;

    var pendingResolve = null;
    var shutdown = false;

    function removeDom() {
       stream.getVideoTracks()[0].stop();
       video.remove();
       div.remove();
       video = null;
       div = null;
       stream = null;
       imgElement = null;
       captureCanvas = null;
       labelElement = null;
    }

    function onAnimationFrame() {
      if (!shutdown) {
        window.requestAnimationFrame(onAnimationFrame);
      }
      if (pendingResolve) {
        var result = "";
        if (!shutdown) {
          captureCanvas.getContext('2d').drawImage(video, 0, 0, 640, 480);
          result = captureCanvas.toDataURL('image/jpeg', 0.8)
        }
        var lp = pendingResolve;
        pendingResolve = null;
        lp(result);
      }
    }

    async function createDom() {
      if (div !== null) {
        return stream;
      }

      div = document.createElement('div');
      div.style.border = '2px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = '600px';
      document.body.appendChild(div);

      video = document.createElement('video');
      video.style.display = 'block';
      video.width = div.clientWidth - 6;
      video.setAttribute('playsinline', '');
      video.onclick = () => { shutdown = true; };
      stream = await navigator.mediaDevices.getUserMedia(
          {video: { facingMode: "environment"}});
      div.appendChild(video);

      imgElement = document.createElement('img');
      imgElement.style.position = 'absolute';
      imgElement.style.zIndex = 1;
      imgElement.onclick = () => { shutdown = true; };
      div.appendChild(imgElement);

      const instruction = document.createElement('div');
      instruction.innerHTML =
          '' +
          'When finished, click here or on the video to stop this demo';
      div.appendChild(instruction);
      instruction.onclick = () => { shutdown = true; };

      video.srcObject = stream;
      await video.play();

      captureCanvas = document.createElement('canvas');
      captureCanvas.width = 640; //video.videoWidth;
      captureCanvas.height = 480; //video.videoHeight;
      window.requestAnimationFrame(onAnimationFrame);

      return stream;
    }
    async function stream_frame(label, imgData) {
      if (shutdown) {
        removeDom();
        shutdown = false;
        return '';
      }

      var preCreate = Date.now();
      stream = await createDom();

      var preShow = Date.now();

      if (imgData != "") {
        var videoRect = video.getClientRects()[0];
        imgElement.style.top = videoRect.top + "px";
        imgElement.style.left = videoRect.left + "px";
        imgElement.style.width = videoRect.width + "px";
        imgElement.style.height = videoRect.height + "px";
        imgElement.src = imgData;
      }

      var preCapture = Date.now();
      var result = await new Promise(function(resolve, reject) {
        pendingResolve = resolve;
      });
      shutdown = false;

      return {'create': preShow - preCreate,
              'show': preCapture - preShow,
              'capture': Date.now() - preCapture,
              'img': result};
    }
    ''')

    display(js)

def video_frame(bbox):
    data = eval_js('stream_frame("{}", "{}")'.format("", bbox))
    return data

In [ ]:
# Memulai streaming video dari webcam
video_stream()

bbox = ''

# Dictionary korespondensi label kelas dan nama ekspresi
label_to_class = {0: "angry", 1: "disgust", 2: "fear", 3: "happy", 4: "neutral", 5: "sad", 6: "surprise"}

while True:
    js_reply = video_frame(bbox)
    if not js_reply:
        break

    img = js_to_image(js_reply["img"])
    bbox_array = np.zeros([480,640,4], dtype=np.uint8)
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    faces = face_cascade.detectMultiScale(gray)

    for (x, y, w, h) in faces:
        face_roi = gray[y:y+h, x:x+w]
        face_roi_resized = cv2.resize(face_roi, (48, 48))
        face_roi_processed = face_roi_resized / 255.0
        face_roi_processed = np.expand_dims(face_roi_processed, axis=0)
        face_roi_processed = np.expand_dims(face_roi_processed, axis=-1)

        # membuat prediksi pada ROI wajah
        prediction = model.predict(face_roi_processed, verbose=0)
        predicted_class_index = np.argmax(prediction)
        predicted_class_label = label_to_class[predicted_class_index]

        # menggambar bounding box di sekitar wajah
        bbox_array = cv2.rectangle(bbox_array, (x, y), (x+w, y+h), (255, 0, 0), 2)

        # menampilkan label yang diprediksi untuk setiap wajah
        cv2.putText(bbox_array, predicted_class_label, (x, y+h+20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)

    bbox_array[:, :, 3] = (bbox_array.max(axis=2) > 0).astype(int) * 255
    bbox_bytes = bbox_to_bytes(bbox_array)
    bbox = bbox_bytes